In [21]:
import numpy as np
import pandas
import scipy
from sklearn.ensemble import RandomForestClassifier, ExtraTreesRegressor
import os
from sklearn.metrics import matthews_corrcoef, recall_score, roc_auc_score, balanced_accuracy_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn import svm
import xgboost
import seaborn as sns
from sklearn.pipeline import Pipeline
from pytorch_tabnet.tab_model import  TabNetClassifier
import torch


In [3]:
os.chdir(r'E:\NNSience\Dwarfs\Brown-Dwarfs-NN\data')

In [14]:
df = pandas.read_csv('data2.csv')

In [15]:
X = df.loc[:, 'PS1imag':'W4mag']
y = df.loc[:, 'label']
cols = df.loc[:, 'PS1imag':'W4mag']

In [22]:
dx = pandas.DataFrame(data=X, columns=cols.columns)
dx['i_z'] = dx['PS1imag']-dx['PS1zmag']
dx['i_y'] = dx['PS1imag']-dx['PS1ymag']
dx['z_y'] = dx['PS1zmag']-dx['PS1ymag']
dx['z_J'] = dx['PS1zmag']-dx['Jmag']
dx['y_J'] = dx['PS1ymag']-dx['Jmag']
dx['J_H'] = dx['Jmag']-dx['Hmag']
dx['H_Ks'] = dx['Hmag']-dx['Ksmag']
dx['W1_W2'] = dx['W1mag']-dx['W2mag']
X_ = np.array(dx.loc[:, 'PS1imag':'W1_W2'])
cols = dx.loc[:, 'PS1imag':'W1_W2']

In [23]:
imputer = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=30, max_features=17, max_depth=20, min_samples_split=15), max_iter=50, random_state=123)
X = imputer.fit_transform(X_)

c:\Users\saser\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [24]:
X_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=123)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=123)

Test of learning rules on L&T data sample with Carnero Rosell et al.(2019) and Burningham et al.(2013)

In [25]:
pos = pandas.DataFrame(data=x_test, columns=cols.columns)
mask =  (pos['z_y']>0.15) & (pos['i_z']>1.2) & (pos['y_J']>1.6) & (pos['PS1zmag']<22) # Carnero Rosell et al.(2019)
mask =  (pos['z_J']>2.5) & (pos['Jmag']<18.8) # Burningham et al.(2013)
corr_dr = matthews_corrcoef(mask, y_test)
print(corr_dr)

0.8731229062271024


Scaling

In [26]:
scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train)
x_val2 = scaler.transform(x_val)
x_test2 = scaler.transform(x_test)

Random Forest, hyperparameters from optuna:

In [27]:
rf_model = RandomForestClassifier(n_estimators=500, max_depth=29, min_samples_leaf=4, max_features= 4)
rf_model.fit(X_train2, y_train)
y_pred_rf = rf_model.predict(x_test2)
matthews_corrcoef(y_test, y_pred_rf)

0.9768605568917293

SVM, hyperparameters from optuna:

In [28]:
svc_model = svm.SVC(kernel='rbf', C=1.1785578339058878, class_weight='balanced', gamma='scale', coef0=4.37386719156764, decision_function_shape='ovr', random_state=123, probability=True)
svc_model.fit(X_train2, y_train)
y_pred_svc=svc_model.predict(x_test2)
matthews_corrcoef(y_test, y_pred_svc)

0.9616256886076121

XGBoost, hyperparameters from optuna:

In [43]:
xgb_clf = xgboost.XGBClassifier(max_depth=24, n_estimators=500, booster='gbtree', learning_rate=0.7643065952721012, gamma=0.60562070482283363, subsample=0.53759136820407905,
                            n_jobs=2, random_state=1, verbosity=0)
xgb_clf.fit(X_train2, y_train)
y_pred_xgb = xgb_clf.predict(x_test2)
matthews_corrcoef(y_pred_xgb, y_test)

c:\Users\saser\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.9741778180983233

In [40]:
dx_train = pandas.DataFrame(X_train2)
dx_train=dx_train.drop(columns=[0,1,2])
dx_test = pandas.DataFrame(x_test2)
dx_test=dx_train.drop(columns=[0,1,2])

In [42]:
xgb_clf2 = xgboost.XGBClassifier(max_depth=24, n_estimators=500, booster='gbtree', learning_rate=0.7643065952721012, gamma=0.60562070482283363, subsample=0.53759136820407905,
                            n_jobs=2, random_state=1, verbosity=0)
xgb_clf2.fit(dx_train, y_train)
y_pred_xgb = xgb_clf2.predict(dx_test)
matthews_corrcoef(y_pred_xgb, y_test)

c:\Users\saser\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.963843605575871

Tabnet:


In [44]:
tbn_clf = TabNetClassifier(n_a=56, n_d=56, n_shared=1, n_steps=1, gamma=1, optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, 
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax')
tbn_clf.fit(X_train2, y_train,            
    eval_set=[(X_train2, y_train), (x_val2, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['balanced_accuracy'],
    max_epochs=1000 , patience=50,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False)

Device used : cpu
epoch 0  | loss: 0.22962 | train_balanced_accuracy: 0.96239 | valid_balanced_accuracy: 0.96307 |  0:00:02s
epoch 1  | loss: 0.06898 | train_balanced_accuracy: 0.96834 | valid_balanced_accuracy: 0.96931 |  0:00:04s
epoch 2  | loss: 0.05854 | train_balanced_accuracy: 0.9758  | valid_balanced_accuracy: 0.98135 |  0:00:07s
epoch 3  | loss: 0.06075 | train_balanced_accuracy: 0.97392 | valid_balanced_accuracy: 0.97814 |  0:00:09s
epoch 4  | loss: 0.06892 | train_balanced_accuracy: 0.97841 | valid_balanced_accuracy: 0.98135 |  0:00:12s
epoch 5  | loss: 0.0577  | train_balanced_accuracy: 0.97718 | valid_balanced_accuracy: 0.97814 |  0:00:14s
epoch 6  | loss: 0.05059 | train_balanced_accuracy: 0.98369 | valid_balanced_accuracy: 0.98412 |  0:00:16s
epoch 7  | loss: 0.06581 | train_balanced_accuracy: 0.97599 | valid_balanced_accuracy: 0.97814 |  0:00:18s
epoch 8  | loss: 0.04319 | train_balanced_accuracy: 0.98231 | valid_balanced_accuracy: 0.98305 |  0:00:20s
epoch 9  | loss: 0.

In [45]:
y_pred_tbn = tbn_clf.predict(x_test2)
corr_tbn = matthews_corrcoef(y_pred_tbn, y_test)
print(corr_tbn)

0.966935394538613


array([ True,  True,  True, False, False,  True, False,  True, False,
        True,  True, False, False, False,  True, False,  True, False,
       False, False, False,  True,  True, False, False,  True,  True,
        True,  True, False,  True,  True, False, False, False, False,
       False,  True, False, False,  True,  True, False, False,  True,
        True,  True,  True, False, False, False, False, False, False,
        True,  True,  True,  True,  True, False, False, False, False,
       False,  True, False, False,  True, False,  True, False, False,
        True,  True, False, False, False,  True,  True, False, False,
        True,  True, False,  True, False, False, False, False, False,
        True, False, False, False, False,  True,  True,  True, False,
       False, False,  True, False, False,  True,  True, False, False,
       False, False,  True, False, False, False,  True, False, False,
        True,  True,  True, False,  True, False,  True, False, False,
       False, False,

In [72]:
def bootstrap_metric(x, 
                     y,
                     samples_cnt = 100,
                     alpha = 0.05,
                     random_state = 42):
    size = len(x)
    np.random.seed(random_state)
    b_metric = np.zeros(samples_cnt)
    for it in range(samples_cnt):
        poses = np.random.choice(x.shape[0], size=x.shape[0], replace=True)
        
        x_boot = x[poses]
        y_boot = y[poses]
        
        #print(x_boot, y_boot)

        m_val = matthews_corrcoef(x_boot, y_boot)
        b_metric[it] = m_val
    
    return b_metric

In [76]:
boot_score_rf = bootstrap_metric(np.array(y_test), y_pred_rf)
boot_score_xgb = bootstrap_metric(np.array(y_test), y_pred_xgb)
boot_score_svc = bootstrap_metric(np.array(y_test), y_pred_svc)
boot_score_tbn = bootstrap_metric(np.array(y_test), y_pred_tbn)

In [ ]:
plt.figure(figsize=(16,6))
sns.boxplot(y=np.concatenate([boot_score_rf, 
                              boot_score_xgb, 
                              boot_score_svc,
                              boot_score_tbn
                             ]),
            x=['RF'] * 1000 + ['XGBoost'] * 1000 + ['SVC'] * 1000 + ["TabNet"] * 1000)